# Approach 2: The Semantic Detective 🕵️‍♀️

# **Section 1: Preprocessing**
+ In this section, we deal with all preprocessing steps required for the rest of this notebook, including importing libraries, installing necessary packages, initializing client module for Google BigQuery etc.
+ Our main tools for this project are `pandas` and `bigquery` from `google.cloud`.
+ Google Cloud's `bigframes` library is uninstalled because of version crashing in Kaggle's default environment.

In [2]:
# Install google-cloud-bigquery-storage for running BigQuery SQL without error
# Add -q to suppress verbose for the sake of readability 
!pip uninstall -q -y bigframes
!pip install -q google-cloud-bigquery-storage

In [3]:
# Import all libraries required for this project
import pandas as pd

from google.cloud import bigquery
from datetime import datetime, timedelta

## Define project and dataset ids
+ To create a database client for BigQuery, a project id is needed: `analog-delight-470708-d0`.
+ We also define dataset and table ids that have been imported from Google Cloud Buckets (GCB) to BigQuery. Please refer to our blog for details of the selected datasets.
    + `steam_game_list` contains the inventory of games available on Steam platform. There are some textual features such as `short description` and `tags` assigned to each game.
    + `review_data` contains players' reviews for some games on Steam.
+ The embedding model `llm_steam` is based on `text-embedding-004`. No further fine-tuning has been performed.

In [4]:
# Initialize BigQuery client with Google Cloud's project id
project_id = 'analog-delight-470708-d0'
client = bigquery.Client(project=project_id)

# We also define dataset and table ids
dataset_id = 'steam'
game_list_data = 'steam_game_list'
review_data = 'steam_reviews'

# We also define the name of text embedding models
embedding_model_name = 'llm_steam'

## Create primary keys for datasets in BigQuery


+ This helper function takes table and column names as arguments to check for the existence of a column in the schema of a table.
+ It is required to avoid repeating costly operations on BigQuery.

In [5]:
# Check whether a column exist in the table schema
def check_column_exists(dataset_id, table_id, name):
    # Given dataset_id and table_id, we retrieve its schema
    table_ref = client.dataset(dataset_id).table(table_id)
    table_schema = client.get_table(table_ref).schema

    # Loop through each field in the schema to determine whether a column exists.
    for field in table_schema:
        if field.name == name:
            return True
    return False

+ We convert *App ID* in `steam.steam_game_list` from string to integer as a new column called *app_id*.
+ This step facilitates table joining with steam's review data `steam.steam_reviews` in BigQuery.

In [6]:
# Check whether the primary key for game_list_data exists
game_list_data_pk = 'app_id'
exist_app_id = check_column_exists(dataset_id, game_list_data, game_list_data_pk)
print('Does the primary key exist? ' + str(exist_app_id))

# If it does not exist, generate it
if not exist_app_id:
    query = f"""
    alter table {project_id}.{dataset_id}.{game_list_data}
    add column if not exists {game_list_data_pk} integer;
    
    update {project_id}.{dataset_id}.{game_list_data}
    set {game_list_data_pk} = cast(`App ID` as integer)
    where true;
    
    alter table {project_id}.{dataset_id}.{game_list_data}
    add primary key ({game_list_data_pk}) not enforced;
    """
    result_pk = client.query(query)
    print(result_pk.result())

Does the primary key exist? True


# **Section 2: Generate Embeddings and Create Vector Indices**

## Generate embeddings for vector search
+ Two embedding vectors are created for the table `steam.steam_game_list` to improve search accuracy. They are based on two textual columns: `short description` and `tags`.
    + `short description` contains the description of the game e.g. "Plunger Knight is a platformer game about a lonely knight who lived in the Middle Age and has lost almost everything he had but had to face a new challenge: robots. In the series of his adventures, he travels the world and time to battle against them."
    + `tags` contains the tags assigned to each game on Steam e.g. `Action: 22, Adventure: 20, Casual: 20, Indie: 20, RPG: 19, Platformer: 12, Funny: 11, Puzzle-Platformer: 10`.

In [7]:
# This function generate embeddings using AI model if not exists
def create_embeddings(embeddings_name, embeddings_model_name, table_name, column_name):
    query = f"""
    alter table `{project_id}.{dataset_id}.{table_name}`
    add column if not exists {embeddings_name} array<float64>;

    update `{project_id}.{dataset_id}.{table_name}` as t
    set t.{embeddings_name} = e.ml_generate_embedding_result
    from (
        select distinct
            ml_generate_embedding_result,
            content
        from ml.generate_embedding(
            model `{project_id}.{dataset_id}.{embeddings_model_name}`,
            (select ifnull({column_name}, ' ') as content
              from `{project_id}.{dataset_id}.{table_name}`
            )
        )
    ) e
    where ifnull(t.{column_name}, ' ') = e.content
    """
    return client.query(query)

The embeddings columns `desc_embeddings` and `tags_embeddings` will be generated if they do not exist in the schema of `steam.steam_game_list` using the above function `create_embeddings()`.
+ `desc_embeddings` is extracted from the textual column `short description`.
+ `tags` is extracted from the textual column `tags`.

In [8]:
# Create text embeddings for 'short description' of each game available on steam
exist_desc = check_column_exists(dataset_id, game_list_data, "desc_embeddings")
print('Does the embeddings for short description of game exist? ' + str(exist_desc))
if not exist_desc:
    result_desc = create_embeddings("desc_embeddings", embedding_model_name, game_list_data, "`Short Description`")
    print(result_desc.result())

# Create text embeddings for 'tags' fof each game available on steam 
exist_tags = check_column_exists(dataset_id, game_list_data, "tags_embeddings")
print('Does the embeddings for tags of game exist? ' + str(exist_tags))
if not exist_tags:
    result_tags = create_embeddings("tags_embeddings", embedding_model_name, game_list_data, "tags")
    print(result_tags.result())

Does the embeddings for short description of game exist? True
Does the embeddings for tags of game exist? True


## Create Vector Index for High-performance Searching
Since the size of review data `steam.steam_reviews` is more than 8 GB, a vector index should be created to speed up the query process for embedding. 
There are in total 3 SQL statments which we will explain as follows:
+ Statement 1 (`create or replace table...`): The review data contains a textual column `review` which is used to create embeddings with AI model.
+ Statement 2 (`create or replace table...`): Empty embeddings i.e. array size of zero are removed by filtering.
+ Statement 3 (`create vector index if not exists ...`): A vector index is created based on non-empty embeddings and stored in `app_id`.

In [9]:
def create_table_reviews_embeddings(out_table_name, in_table_name):
    query = f"""
    create or replace table `{project_id}.{dataset_id}.{out_table_name}` as
    select *
    from ml.generate_embeddings(
    model `{project_id}.{dataset_id}.{embedding_model_name}`,
    (select Review as content, app_id from `{project_id}.{dataset_id}.{in_table_name}`),
    struct(
        true AS flatten_json_output, 
        'RETRIEVAL_DOCUMENT' as task_type
        )
    );

    create or replace table {dataset_id}.{out_table_name}_notnull as 
    select * from {dataset_id}.{out_table_name} 
    where ARRAY_LENGTH(ml_generate_embedding_result) > 0;

    create vector index if not exists embedding_index
    on {dataset_id}.{out_table_name}_notnull(ml_generate_embedding_result)
    storing(app_id)
    options(distance_type='COSINE', index_type='IVF');
    """
    return client.query(query)

Since it is a costly process with more than 21.5 million records, we run the above query only for the first time.

In [10]:
table_review_embeddings = 'review_embeddings'
try:
    client.dataset(dataset_id).table(table_review_embeddings)
    print(f'Table {table_review_embeddings} already exists.')
except: 
    print(f'Table {table_review_embeddings} does not exist. A new table will be created.')
    handler = create_table_reviews_embeddings('review_embeddings', review_data)

Table review_embeddings already exists.


# **Section 3: Usecases of Google BigQuery AI in Product Positioning**

## Usecase 3.1 - Search a list of similar Steam games given a user query on game characteristics

The function `get_list_of_games` wraps the SQL query for retrieving games which are close to the user query in semantic. 
For example, a user query can be "`I would like to find a multi-person strategic game on farming in an open-world setting.`". 
To enhance accuracy, this SQL can select games with both description and tags close to the user query via an inner join on `app_id`.

*Parameters to tune:*
+ `number_of_games`: This parameter controls `top_k` in vector search on both embeddings. For the result to be accurate, the same semantic meaning should appear on both description and tags fields.
+ `min_reviews`: This parameter controls the popularity of games by the number of reviews they received, regardless of positive or negative.

*Expected results:*
A dataframe should be returned from `get_list_of_games` containing all columns in `steam.steam_game_list` for which the semantic meaning is close to the user query.

In [11]:
def get_list_of_games(user_input, number_of_games, min_reviews):
    embeddings = ["desc_embeddings", "tags_embeddings"]
    query = f"""
    select a.base.*
    from vector_search(
        (select {embeddings[0]}, name, app_id, `short description`, tags, `positive reviews`, `negative reviews` 
        from `{project_id}.{dataset_id}.{game_list_data}`
        where (`positive reviews` > {min_reviews}) or (`negative reviews` > {min_reviews})),
        '{embeddings[0]}',
        (select ml_generate_embedding_result, content as query 
        from ml.generate_embedding(
        model `{project_id}.{dataset_id}.{embedding_model_name}`,
            (select '{user_input}' as content))
        ),
        top_k => {number_of_games},
        distance_type => 'COSINE') as a
    inner join 
    vector_search(
        (select {embeddings[1]}, name, app_id, `short description`, tags, `positive reviews`, `negative reviews` 
        from `{project_id}.{dataset_id}.{game_list_data}`
        where (`positive reviews` > {min_reviews}) or (`negative reviews` > {min_reviews})),
        '{embeddings[1]}',
        (select ml_generate_embedding_result, content as query 
        from ml.generate_embedding(
        model `{project_id}.{dataset_id}.{embedding_model_name}`,
            (select '{user_input}' as content))
        ),
        top_k => {number_of_games},
        distance_type => 'COSINE') as b
        on a.base.app_id = b.base.app_id
    """
    df = client.query(query).to_dataframe()
    return df

As an example, we demonstrate the above query with the following examples

+ `I would like to find a multi-person strategic game on farming in an open-world setting.`
+ `I would like to find a multi-person action RPG game with dragon`

First, we retrieve data based on the above query strings.

In [54]:
# Parameters to set
number_of_games = 100
min_reviews = 1000

# Example 1
user_input = "I would like to find a multi-person strategic game on farming in an open-world setting."
df_retrieve1 = get_list_of_games(user_input, number_of_games, min_reviews)

# Example 2
user_input = "I would like to find a multi-person action RPG game with dragon"
df_retrieve2 = get_list_of_games(user_input, number_of_games, min_reviews)

Take a look of the relevant game ids. There are 12 games satisfying the first query and 2 games satisfying the second query.

In [55]:
# Convert pd.Series to python list object for postprocessing 
app_ids_1 = df_retrieve['app_id'].values.tolist()
print('The list of relevant games (app_ids) retrieved by the user query: ', app_ids_1)

# Convert pd.Series to python list object for postprocessing 
app_ids_2 = df_retrieve2['app_id'].values.tolist()
print('The list of relevant games (app_ids) retrieved by the user query: ', app_ids_2)

The list of relevant games (app_ids) retrieved by the user query:  [220260, 673950, 405710, 1248130, 1432860, 534370, 678900, 704030, 787860, 1123830, 495560, 840010]
The list of relevant games (app_ids) retrieved by the user query:  [263500, 487000]


Separately, we should calculate the metric for good rating to facilitate our analysis. The formula is to divide the number of positive reviews by the number of negative reviews, which yields an odd for comparison. Finally, we sort the above dataset by this metric.

+ Query 1: The leading two highly-rated games are `Farm Together` and `Farming Simulator 19`, and the worst game is `Professional Farmer: Cattle and Crops`.
+ Query 2: `Quest Hunter` is ranked over `Dragons and Titans`.

In [60]:
# Calculate the metric column to measure good rating and sort the dataset in descending order.
df_retrieve1['odd'] = df_retrieve1['positive reviews']/df_retrieve1['negative reviews']
df_sort = df_retrieve.iloc[:, 1:].sort_values('odd', ascending=False)
df_sort

,name,app_id,short description,tags,positive reviews,negative reviews,odd
1,Farm Together,673950,"Grow your own farm all by yourself, or cooperate with your friends in this unique, relaxing farming experience!","Agriculture: 362, Multiplayer: 338, Simulation: 280, Casual: 255, Relaxing: 253, Online Co-Op: 242, Building: 230, Farming Sim: 226, Co-op: 205, Cute: 186, Sandbox: 175, Singleplayer: 165, Character Customization: 159, Third Person: 150, Cartoony: 149, Replay Value: 146, Controller: 142, Colony Sim: 136, Female Protagonist: 135, Horses: 134",17160,1070,16.037383
8,Farming Simulator 19,787860,"The best-selling franchise takes a giant leap forward with a complete overhaul of the graphics engine, offering the most striking and immersive visuals and effects, along with the deepest and most complete farming experience ever.","Simulation: 834, Farming Sim: 738, Multiplayer: 538, Agriculture: 529, Realistic: 436, Open World: 401, Driving: 398, Management: 344, Singleplayer: 344, Automobile Sim: 338, Moddable: 320, Co-op: 302, Resource Management: 264, Relaxing: 254, First-Person: 190, Online Co-Op: 174, Economy: 157, Family Friendly: 133, Atmospheric: 128, Strategy: 128",60886,3847,15.826878
0,Farming Simulator 2013 Titanium Edition,220260,"Animal husbandry, crops, sales… It's up to you to manage and grow your own farm in a huge, totally new world!","Simulation: 513, Farming Sim: 509, Multiplayer: 167, Casual: 165, Farming: 142, Open World: 135, Relaxing: 108, Co-op: 75, Moddable: 71, Management: 70, Physics: 70, Agriculture: 66, Singleplayer: 65, Realistic: 53, Online Co-Op: 39, Great Soundtrack: 33, Sandbox: 33, Driving: 26, Atmospheric: 24, Funny: 21",4529,396,11.436869
11,Garden Paws,840010,You have inherited your grandparents farm as they set off to travel the world. Work with Frank at the local construction shop to finish off the final touches on your new home. Aid Frank and Mayor Wilson in building up the town so it may reach it's full potential.,"Exploration: 184, Sandbox: 178, Agriculture: 173, Building: 168, Crafting: 162, Farming Sim: 153, RPG: 151, Cute: 149, Relaxing: 148, Open World: 147, Casual: 142, Family Friendly: 141, Simulation: 139, Cartoony: 138, Co-op: 134, Moddable: 132, Wholesome: 128, Multiplayer: 120, Online Co-Op: 118, Cozy: 107",1970,184,10.706522
4,Sun Haven,1432860,"Build your farm and relationships with townsfolk, or forge ahead on a quest of magic, monsters, and dragons. Level up through a skill tree in 8 person multiplayer, or adventure solo!","Early Access: 362, Farming Sim: 329, Pixel Graphics: 322, Multiplayer: 314, Life Sim: 313, RPG: 303, Character Customization: 291, Agriculture: 282, Relaxing: 274, Simulation: 271, Dating Sim: 268, Crafting: 260, Singleplayer: 257, Adventure: 256, Magic: 249, Fantasy: 236, Mining: 233, Fishing: 229, Combat: 228, Exploration: 224",2672,251,10.645418
3,Farming Simulator 22,1248130,"Create your farm and let the good times grow! Harvest crops, tend to animals, manage productions, and take on seasonal challenges.","Simulation: 644, Co-op: 628, Farming Sim: 625, Agriculture: 601, Moddable: 571, Management: 551, Sandbox: 512, Multiplayer: 511, 3D: 488, Immersive Sim: 486, First-Person: 485, Character Customization: 458, Economy: 455, Online Co-Op: 452, Family Friendly: 445, Open World: 441, Third Person: 438, Nature: 427, America: 419, Horses: 418",33215,3797,8.747696
2,Staxel,405710,"Grow your farm, meet the villagers, and join your friends online in building your world.","Farming Sim: 264, Cute: 248, Character Customization: 245, Agriculture: 232, Nature: 226, Sandbox: 223, Relaxing: 217, Building: 210, Open World: 205, Voxel: 200, Pixel Graphics: 195, Online Co-Op: 194, Casual: 191, RPG: 183, Simulation: 180, Multiplayer: 174, Adventure: 168, Co-op: 166, Singleplayer: 127, Survival: 118",3451,860,4.012791
9,Farm Manager 2021,1123830,"Get ready for a logistic challenge in the new and even better Farm Manager 2021! Plan your field work according to the seaso

In [61]:
df_retrieve2['odd'] = df_retrieve2['positive reviews']/df_retrieve2['negative reviews']
df_sort2 = df_retrieve2.iloc[:, 1:].sort_values('odd', ascending=False)
df_sort2

,name,app_id,short description,tags,positive reviews,negative reviews,odd
1,Quest Hunter,487000,"Quest Hunter is an isometric action-RPG where your choice drives the story. Find tons of treasures and secrets, solve puzzles, equip your character, level up your skills and finish off enemy bosses. Play on the couch, alone or with friends, or make it an online party for up to four players!","Character Customization: 604, Isometric: 593, Cartoon: 526, Exploration: 490, Party-Based RPG: 483, Relaxing: 478, Fantasy: 474, Family Friendly: 470, RPG: 458, Action: 454, Cute: 452, Hack and Slash: 442, Funny: 438, Controller: 436, Adventure: 432, Co-op: 432, Story Rich: 423, Magic: 412, Dungeon Crawler: 399, Rogue-like: 393",1540,340,4.529412
0,Dragons and Titans,263500,"Dragons and Titans is a fast paced MOBA with classic RPG elements. Select from over 30 unique dragons and 30 legendary weapons. Upgrade dragons and forge weapons to unlock more power. Free your Titan in 5v5 PvP battles across 3 different map types or play in ‘Adventure Mode’, traveling across different regions and unlocking rewards.","Free to Play: 507, Dragons: 339, MOBA: 262, Multiplayer: 222, RPG: 215, Action: 165, Adventure: 165, Singleplayer: 112, Fantasy: 101, Co-op: 100, Indie: 100, Massively Multiplayer: 97, PvP: 97, Strategy: 94, Survival: 65, MMORPG: 57, Open World: 50, FPS: 36, Sandbox: 27, Atmospheric: 20",2686,750,3.581333


## Usecase 3.2 - Search relevant reviews for the list of games given a query on product features

The product features can be systematically analysed by querying the review data of the above games. Using the first case, farming games, as an example, we input 3 queries to measure age, music and price aspects of the games.
+ `Is this game easy to play for elderly?`
+ `Tell me the music and visual aspects of this game.`
+ `Is the price expensive or cheap or just affordable given the quality?`

The review data is restricted to the above game ids i.e. `app_ids_1` and we run the below function `search_reviews` iteratively for each query.

*Parameters to tune*:
+ `number_of_reviews`: This parameter controls the number of games to be displayed.

In [16]:
number_of_reviews = 20
user_input_age = 'Is this game easy to play for elderly?'
user_input_music = 'Tell me the music and visual aspects of this game.'
user_input_price = 'Is the price expensive or cheap or just affordable given the quality?'

def search_reviews(user_input, number_of_reviews):
    query = f"""
    select a.base.*
    from vector_search(
        (select ml_generate_embedding_result, app_id, content
        from `{project_id}.{dataset_id}.{table_review_embeddings}_notnull`
        where app_id in ({','.join([str(app_id) for app_id in app_ids_1])})),
        'ml_generate_embedding_result',
        (select ml_generate_embedding_result, content as query 
        from ml.generate_embedding(
        model `{project_id}.{dataset_id}.{embedding_model_name}`,
            (select '{user_input}' as content))
        ),
        top_k => {number_of_reviews},
        distance_type => 'COSINE') a
    """
    df = client.query(query).to_dataframe()
    return df

In [62]:
df_age = search_reviews(user_input_age, number_of_reviews)
df_music = search_reviews(user_input_music, number_of_reviews)
df_price = search_reviews(user_input_price, number_of_reviews)

For analytic purpose, it is better to merge with other attributes of the game based on `app_id`.

In [63]:
def merge_dataset(df):
    return df.iloc[:, 1:].\
        merge(df_retrieve[['app_id', 'name', 'odd']], on='app_id').\
        sort_values('name')

### Is this game easy to play for elderly?
+ `Farm Together` and `Farming Simulator 19` with high good-to-bad reviews ratio is commented as friendly game to kids and elderly. So, the ease to play for all ages with is an important factor for farming game.

In [64]:
pd.set_option('display.max_colwidth', None)
print(f'Analysis: {user_input_age}')
merge_dataset(df_age)

Analysis: Is this game easy to play for elderly?


,app_id,content,name,odd
0,673950,"Cute little game, easy to play even with limited mobility. :)",Farm Together,16.037383
17,673950,Fun game. Easy Friendly,Farm Together,16.037383
15,673950,Super fun and cool game to play - something easy and doesn't take much effort.,Farm Together,16.037383
14,673950,This is a fun game that anyone could like.,Farm Together,16.037383
13,673950,This game is so fun for adults and kids,Farm Together,16.037383
11,673950,True casual game! Easy to play and very enjoyable.,Farm Together,16.037383
8,673950,"Very casual, looks good, fun and easy. Great for players of all ages.",Farm Together,16.037383
19,673950,This is a super fun game to play and super relaxing. It's also great for a family that games together!,Farm Together,16.037383
4,673950,"I like this game for its stress-free and non-violent environment. I think it's a good game for all age groups that are capable of playing video games. For people who are looking for a high intensity game this would not be the game for them. But for anyone seeking a ""G-rated"" and low to no stress game this one is awesome!",Farm Together,16.037383
6,787860,great game for all ages . my family loves this game,Farming Simulator 19,15.826878


### Tell me the music and visual aspects of this game.
+ Both games with high or low good-to-bad reviews ratio may have good music and visual aspects. The result shows that music and visual aspects is not the most important factor.

In [65]:
print(f'Analysis: {user_input_music}')
merge_dataset(df_music)

Analysis: Tell me the music and visual aspects of this game.


app_id  \
9  495560   
0  673950   
1  673950   
2  673950   
3  787860   
4  405710   
5  405710   
6  405710   
7  405710   
8  405710   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# Is the price expensive or cheap or just affordable given the quality?
+ The low ranking game `Staxel` is found to be over-priced.
+ For high ranking games `Farm Together` and `Farming Simulator 19`, players are willing to spend money on the quality.

In [66]:
print(f'Analysis: {user_input_price}')
merge_dataset(df_price)

Analysis: Is the price expensive or cheap or just affordable given the quality?


,app_id,content,name,odd
19,673950,Better than i expected,Farm Together,16.037383
17,787860,"Expected more, its okay but now my type want refund",Farming Simulator 19,15.826878
16,787860,"nice graphics, good prices.",Farming Simulator 19,15.826878
14,787860,Got it on sale worth every penny,Farming Simulator 19,15.826878
13,787860,Really good quality,Farming Simulator 19,15.826878
12,787860,"It's not much, but it's honest work\n\nGreat for chilling with your buds",Farming Simulator 19,15.826878
11,787860,good quality\n,Farming Simulator 19,15.826878
10,787860,"Good Start, Now make it better. :P worth the money though\n",Farming Simulator 19,15.826878
18,787860,Great exspreise,Farming Simulator 19,15.826878
0,787860,it is worth the money\n,Farming Simulator 19,15.826878
